![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [97]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [98]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [99]:
#1: student_id Column
#Columns containing integers only must be stored as 32-bit integers (int32).
ds_jobs_transformed["student_id"] = ds_jobs_transformed["student_id"].astype("int32")

In [100]:
#2: city Column
ds_jobs_transformed["city"].unique()
ds_jobs_transformed["city"] = ds_jobs_transformed["city"].astype("category")

In [101]:
#3: city_development_index  Column
ds_jobs_transformed["city_development_index"].unique()
ds_jobs_transformed["city_development_index"] = ds_jobs_transformed["city_development_index"].astype("float16")

In [102]:
#4: gender Column
ds_jobs_transformed["gender"] = ds_jobs_transformed["gender"].astype("category")
ds_jobs_transformed["gender"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: gender
Non-Null Count  Dtype   
--------------  -----   
14650 non-null  category
dtypes: category(1)
memory usage: 19.0 KB


In [103]:
#5: relevant_experience Column
ds_jobs_transformed["relevant_experience"].unique()
Order = {"Has relevant experience": True, "No relevant experience": False}
ds_jobs_transformed["relevant_experience"] = ds_jobs_transformed["relevant_experience"].map(Order)

In [104]:
#6: enrolled_university column
print(ds_jobs_transformed["enrolled_university"].value_counts())
order = ["no_enrollment", "Part time course", "Full time course"]
category = pd.CategoricalDtype(order, ordered=True)
ds_jobs_transformed["enrolled_university"] = ds_jobs_transformed["enrolled_university"].astype(category)

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64


In [105]:
#7: education_level Column
print(ds_jobs_transformed["education_level"].value_counts())
order = ["Primary School", "High School", "Graduate", "Masters", "Phd"]
category = pd.CategoricalDtype(order, ordered=True)
ds_jobs_transformed["education_level"] = ds_jobs_transformed["education_level"].astype(category)

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64


In [106]:
#8: major_discipline Column
print(ds_jobs_transformed["major_discipline"].value_counts())
ds_jobs_transformed["major_discipline"] = ds_jobs_transformed["major_discipline"].astype("category")
ds_jobs_transformed["major_discipline"].info()

STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64
<class 'pandas.core.series.Series'>
RangeIndex: 19158 entries, 0 to 19157
Series name: major_discipline
Non-Null Count  Dtype   
--------------  -----   
16345 non-null  category
dtypes: category(1)
memory usage: 19.0 KB


In [107]:
#9: experience Column
print(ds_jobs_transformed["experience"].value_counts())
order = ["<1"]+ list(map(str, range(1, 21))) + [">20"]
category = pd.CategoricalDtype(order, ordered=True)
ds_jobs_transformed["experience"] = ds_jobs_transformed["experience"].astype(category)

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64


In [108]:
#10: company_size Column
print(ds_jobs_transformed["company_size"].value_counts())
order = ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"]
category = pd.CategoricalDtype(order, ordered=True)
ds_jobs_transformed["company_size"] = ds_jobs_transformed["company_size"].astype(category)

50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64


In [109]:
#11: company_type Column
print(ds_jobs_transformed["company_type"].value_counts())
ds_jobs_transformed["company_type"] = ds_jobs_transformed["company_type"].astype("category")

Pvt Ltd                9817
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: company_type, dtype: int64


In [110]:
#12: last_new_job Column
print(ds_jobs_transformed["last_new_job"].value_counts())
order = ["never", "1", "2", "3", "4", ">4"]
category = pd.CategoricalDtype(order, ordered=True)
ds_jobs_transformed["last_new_job"] = ds_jobs_transformed["last_new_job"].astype(category)

1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: last_new_job, dtype: int64


In [111]:
#13: training_hours Column
print(ds_jobs_transformed["training_hours"].value_counts())
ds_jobs_transformed["training_hours"] = ds_jobs_transformed["training_hours"].astype("int32")

28     329
12     292
18     291
22     282
50     279
      ... 
266      6
234      5
272      5
286      5
238      4
Name: training_hours, Length: 241, dtype: int64


In [112]:
#14: job_change column
print(ds_jobs_transformed["job_change"].value_counts())
Order = {0.0: False, 1.0: True}
ds_jobs_transformed["job_change"] = ds_jobs_transformed["job_change"].map(Order)

0.0    14381
1.0     4777
Name: job_change, dtype: int64


In [113]:
#filter for students with 10 or more years of experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed["experience"] >= "10") & (ds_jobs_transformed["company_size"]>="1000-4999")]